In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

In [2]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['test.csv',
 'structures',
 'sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'structures.csv',
 'train.csv']

In [4]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if (f.endswith('.pkl')) and (not f.startswith('.'))]

['giba-r-data-table-simple-features-1-17-lb_train.pkl',
 'eem_train.pkl',
 'coulomb-interaction-speed-up_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_test.pkl',
 'keras-neural-net-for-champs_train.pkl',
 'brute-force-feature-engineering_test.pkl',
 'angles-and-distances_test.pkl',
 'giba-r-data-table-simple-features-0-991-lb_test.pkl',
 'keras-neural-net-for-champs_test.pkl',
 'brute-force-feature-engineering-mini_train.pkl',
 'laplaction_matrix_test.pkl',
 'dataset-with-number-of-bonds-between-atoms_train.pkl',
 'coulomb-interaction-speed-up_test.pkl',
 'bonds-from-structure-data_train.pkl',
 'laplaction_matrix_train.pkl',
 'giba-r-data-table-simple-features-0-991-lb_train.pkl',
 'eem_test.pkl',
 'brute-force-feature-engineering_train.pkl',
 'molecule-with-openbabel_train.pkl',
 'molecular-properties-eda-and-models_train.pkl',
 'brute-force-feature-engineering-mini_test.pkl',
 'eachtype_train.pkl',
 'molecular-properties-eda-and-models_test.pkl',
 'bonds-from-structure-dat

In [5]:
mytrial = []

In [ ]:
"""
y=scalar_coupling_constant, type=0
257
{'columns': ['cos_center1', 'adC3', 'dist_H_4_y', 'inv_dist1R', 'eem_1', 'dist_C_0_y', 'dist_to_type_std', 'cos_f0', 'molecule_dist_min', 'inv_dist1', 'eem_0', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_std_diff', 'tertiary_angle_0', 'tertiary_distance_5', 'dist_O_1_x', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_4_y', 'dist_to_type_mean', 'inv_distPR', 'inv_dist1E', 'adC1', 'dist_C_0_x', 'dist_H_1_x', 'dist_O_0_x', 'adN1', 'dist_to_type_1_mean', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'dist_H_3_x', 'tertiary_angle_1', 'linkM0', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'molecule_type_dist_mean_diff', 'dist_to_type_0_mean', 'dist_C_3_x', 'tertiary_distance_4', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'dist_H_0_y', 'tertiary_distance_0', 'dist_C_2_x', 'tertiary_angle_5', 'dist_H_0_x', 'coulomb_H.y', 'tertiary_distance_1', 'dist_H_2_y', 'dist_O_1_y', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'distC1', 'dist_xyz', 'dist_C_2_y', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'distC0', 'vander_H.x', 'dist_H_2_x', 'adC2', 'yukawa_H.y', 'molecule_atom_index_1_dist_min_diff', 'cos_center0', 'dist_C_4_x', 'dist_N_0_x', 'atom_index_farthest_0', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=1
454
{'columns': ['min_molecule_atom_1_dist_xyz', 'tertiary_distance_1', 'adC3', 'dist_H_2_y', 'inv_dist1R', 'dist_C_0_y', 'cos_f0', 'atom_1_bond_lengths_std', 'inv_dist1', 'eem_0', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'cos_center0_center1', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'distance_center0', 'distN0', 'vander_N.x', 'inv_distPR', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'adC4', 'dist_C_0_x', 'dist_H_1_x', 'tertiary_angle_3', 'dist_O_0_x', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'molecule_atom_index_1_dist_max_div', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'dist_C_1_x', 'vander_H.x', 'dist_to_type_0_mean', 'inv_distP', 'dist_C_3_x', 'tertiary_distance_4', 'bond_atom', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'vander_H.y', 'dist_N_1_x', 'dist_H_0_y', 'yukawa_H.y', 'tertiary_distance_0', 'yukawa_H.x', 'dist_C_2_x', 'linkN', 'dist_N_0_x', 'dist_H_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=2
681
{'columns': ['tertiary_distance_6', 'adC3', 'inv_dist1R', 'eem_1', 'cos_f0', 'dist_C_0_y', 'cos_c1', 'yukawa_O.x', 'inv_dist1', 'tertiary_atom_2', 'molecule_atom_index_0_dist_max_div', 'tertiary_distance_2', 'tertiary_angle_0', 'dist_O_1_x', 'tertiary_distance_3', 'mean_molecule_atom_0_dist_xyz', 'inv_distPR', 'min_molecule_atom_0_dist_xyz', 'molecule_type_dist_max', 'dist_C_0_x', 'dist_H_1_x', 'adC1', 'dist_O_0_x', 'molecule_atom_index_0_dist_max_diff', 'adN1', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'inv_dist0', 'dist_C_3_x', 'atom_0_bond_lengths_max', 'atom_1_bond_lengths_min', 'bond_atom', 'inv_dist0R', 'dist_H_0_y', 'molecule_atom_index_1_dist_min_div', 'dist_C_2_x', 'cos_c0', 'dist_H_0_x', 'tertiary_distance_1', 'dist_H_2_y', 'tertiary_atom_3', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'vander_O.y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'yukawa_O.y', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_2', 'tertiary_atom_0', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'atom_1_bond_lengths_max', 'dist_C_1_y', 'distC0', 'adC2', 'tertiary_atom_4', 'cos_c0_c1', 'atom_index_1_n_cycle', 'dist_N_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=3
139
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=4
219
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=5
299
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=6
379
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=7
449
{'columns': ['max_molecule_atom_0_dist_xyz', 'tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_1_dist_min_diff', 'dist_C_4_x', 'adC3', 'distN0', 'atom_0_bond_lengths_max', 'dist_C_3_y', 'tertiary_angle_1', 'atom_1_bond_lengths_mean', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'molecule_atom_index_1_dist_max_div', 'molecule_atom_index_0_dist_min_div', 'dist_C_0_x', 'adC1', 'tertiary_distance_1', 'atom_1_bond_lengths_std', 'dist_H_0_y', 'dist_xyz', 'dist_O_0_x', 'inv_distPR', 'cos_c0_c1', 'cos_c0', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_C_3_x', 'atom_1_n_bonds', 'bond_atom', 'dist_C_2_x', 'dist_N_0_y', 'dist_N_0_x', 'atom_index_1_cycle_size_mean', 'min_molecule_atom_0_dist_xyz', 'tertiary_angle_3', 'tertiary_distance_3', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
"""

y=scalar_coupling_constant, type=0
257
{'columns': ['cos_center1', 'adC3', 'dist_H_4_y', 'inv_dist1R', 'eem_1', 'dist_C_0_y', 'dist_to_type_std', 'cos_f0', 'molecule_dist_min', 'inv_dist1', 'eem_0', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_std_diff', 'tertiary_angle_0', 'tertiary_distance_5', 'dist_O_1_x', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_4_y', 'dist_to_type_mean', 'inv_distPR', 'inv_dist1E', 'adC1', 'dist_C_0_x', 'dist_H_1_x', 'dist_O_0_x', 'adN1', 'dist_to_type_1_mean', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'dist_H_3_x', 'tertiary_angle_1', 'linkM0', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'molecule_type_dist_mean_diff', 'dist_to_type_0_mean', 'dist_C_3_x', 'tertiary_distance_4', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'dist_H_0_y', 'tertiary_distance_0', 'dist_C_2_x', 'tertiary_angle_5', 'dist_H_0_x', 'coulomb_H.y', 'tertiary_distance_1', 'dist_H_2_y', 'dist_O_1_y', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dis

In [6]:
# , 'fc', 'sd','pso','dso'
for y in ['scalar_coupling_constant']:
    for t in np.arange(3,8,1):
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
#         df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
        
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
        n_samples = 500000 if df_train.shape[0] > 500000 else df_train.shape[0]
        df_train = df_train.sample(n_samples).reset_index(drop=True)
#         df_test = df_test[df_test['type']==t].reset_index(drop=True)
        df_test = pd.DataFrame()
        

        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'KFold',
                'init':{
                    'n_splits': 5,
                    'shuffle': True,
                    'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls': 'lgb.LGBMRegressor',
                'init': {
                    'learning_rate': 0.2833769330240482,
                    'feature_fraction': 0.8818248470204605,
                    'bagging_fraction': 0.8205197060908092,
                    'min_data_in_leaf': 202,
                    'lambda_l1': 0.017039063121824582,
                    'lambda_l2': 0.8318702431636841,
                    'max_bin': 100,
                    'num_leaves': 255,
                    'random_state': 3895,
                    'n_jobs': 16
                },
                'fit': {}
            },
            'metric': 'mean_absolute_error'
        }



        process_queue = PQueue(df_train, df_test, param, mytrial)
        sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
        select_topn = RFESelectTopNProcess(**{'n_features_remain':20, 'n_features_to_remove':10})
        remove_useless = RFERemoveUselessFeaturesProcess(**{})
        process_queue.insert_node(sort_features)
        process_queue.insert_node(select_topn)
        process_queue.insert_node(remove_useless)

        try:
            result = process_queue.run()
        except Exception as e:
            print(e.__str__())
        print(len(process_queue.trial))
        print(process_queue.param)
    

y=scalar_coupling_constant, type=3
139
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_dis

In [9]:
for param in columns_list:
    print(len(param['columns']))

78
58
68
59
59
59
59
49


In [6]:
def objective(trial):
        
    learning_rate = trial.suggest_uniform('learning_rate', .01, .5)
    feature_fraction = trial.suggest_uniform('feature_fraction', .6, 1)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.6, 1)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 200, 800)
    lambda_l1 = trial.suggest_loguniform('lambda_l1', 1e-6, 1e2)
    lambda_l2 = trial.suggest_loguniform('lambda_l2', 1e-6, 1e2)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    num_leaves = trial.suggest_int('num_leaves', 4, 512)
        
    args = columns_list[t].copy()
    args['model']['init']={
                'learning_rate':learning_rate,
                'feature_fraction':feature_fraction,
                'bagging_fraction':bagging_fraction,
                'min_data_in_leaf':min_data_in_leaf,
                'lambda_l1':lambda_l1,
                'lambda_l2':lambda_l2,
                'max_bin':max_bin,
                'num_leaves':num_leaves,
                'n_jobs':16
    }
    
    df_his, df_feature_importances, df_valid_pred, df_test_pred =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=None, trial=mytrial, is_output_feature_importance=False, trial_level=1)
    val_metric_mean = np.mean(df_his.valid)
    return val_metric_mean



In [23]:
df_trial = pd.DataFrame(mytrial)
df_trial[df_trial['message']=='tune hyperparam'][['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff']].sort_values(by=['val_metric_mean']).head()

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff
212,2019-07-08 14:06:29.634383,tune hyperparam,38,0.863236,0.930069,0.066833
215,2019-07-08 14:08:20.456876,tune hyperparam,38,0.862214,0.930458,0.068244
234,2019-07-08 14:18:47.324132,tune hyperparam,38,0.840667,0.932579,0.091912
202,2019-07-08 14:00:26.322634,tune hyperparam,38,0.855637,0.932970,0.077333
145,2019-07-08 13:30:42.870039,tune hyperparam,38,0.866630,0.936855,0.070225


In [7]:
df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
df_train['y'] = df_train['scalar_coupling_constant']

In [8]:
oof_file_folder =  '../../data/oof'
[f for f in os.listdir(oof_file_folder) if f.startswith('lgbm')]

['lgbm_sd_test.pkl',
 'lgbm_fc_train.pkl',
 'lgbm_pso_train.pkl',
 'lgbm_pso_test.pkl',
 'lgbm_fc_test.pkl',
 'lgbm_sd_train.pkl']

In [9]:
df_oof_train = pd.DataFrame()
df_oof_test = pd.DataFrame()
for f in [f for f in os.listdir(oof_file_folder) if f.startswith('lgbm')]:
    feat_name = f.split('_')[1]
    if 'train' in f:
        df_oof_i = pd.read_pickle(f'{oof_file_folder}/{f}')[['id', feat_name]].rename(columns={'id':'index', feat_name:f'oof_{feat_name}'})
        df_oof_train = pd.concat([df_oof_train, df_oof_i], axis=1)
    if 'test' in f:
        df_oof_i = pd.read_pickle(f'{oof_file_folder}/{f}')[['id', feat_name]].rename(columns={'id':'index', feat_name:f'oof_{feat_name}'})
        df_oof_test = pd.concat([df_oof_test, df_oof_i], axis=1)

In [10]:
df_train = pd.concat([df_train, df_oof_train[['oof_sd','oof_fc','oof_pso']]], axis=1)
df_test = pd.concat([df_test, df_oof_test[['oof_sd','oof_fc','oof_pso']]], axis=1)

In [13]:
columns_list = [{'columns': ['oof_sd','oof_fc','oof_pso','cos_center1', 'adC3', 'dist_H_4_y', 'inv_dist1R', 'eem_1', 'dist_C_0_y', 'dist_to_type_std', 'cos_f0', 'molecule_dist_min', 'inv_dist1', 'eem_0', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_std_diff', 'tertiary_angle_0', 'tertiary_distance_5', 'dist_O_1_x', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_4_y', 'dist_to_type_mean', 'inv_distPR', 'inv_dist1E', 'adC1', 'dist_C_0_x', 'dist_H_1_x', 'dist_O_0_x', 'adN1', 'dist_to_type_1_mean', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'dist_H_3_x', 'tertiary_angle_1', 'linkM0', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'molecule_type_dist_mean_diff', 'dist_to_type_0_mean', 'dist_C_3_x', 'tertiary_distance_4', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'dist_H_0_y', 'tertiary_distance_0', 'dist_C_2_x', 'tertiary_angle_5', 'dist_H_0_x', 'coulomb_H.y', 'tertiary_distance_1', 'dist_H_2_y', 'dist_O_1_y', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'distC1', 'dist_xyz', 'dist_C_2_y', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'distC0', 'vander_H.x', 'dist_H_2_x', 'adC2', 'yukawa_H.y', 'molecule_atom_index_1_dist_min_diff', 'cos_center0', 'dist_C_4_x', 'dist_N_0_x', 'atom_index_farthest_0', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_sd','oof_fc','oof_pso','min_molecule_atom_1_dist_xyz', 'tertiary_distance_1', 'adC3', 'dist_H_2_y', 'inv_dist1R', 'dist_C_0_y', 'cos_f0', 'atom_1_bond_lengths_std', 'inv_dist1', 'eem_0', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'cos_center0_center1', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'distance_center0', 'distN0', 'vander_N.x', 'inv_distPR', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'adC4', 'dist_C_0_x', 'dist_H_1_x', 'tertiary_angle_3', 'dist_O_0_x', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'molecule_atom_index_1_dist_max_div', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'dist_C_1_x', 'vander_H.x', 'dist_to_type_0_mean', 'inv_distP', 'dist_C_3_x', 'tertiary_distance_4', 'bond_atom', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'vander_H.y', 'dist_N_1_x', 'dist_H_0_y', 'yukawa_H.y', 'tertiary_distance_0', 'yukawa_H.x', 'dist_C_2_x', 'linkN', 'dist_N_0_x', 'dist_H_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_sd','oof_fc','oof_pso','tertiary_distance_6', 'adC3', 'inv_dist1R', 'eem_1', 'cos_f0', 'dist_C_0_y', 'cos_c1', 'yukawa_O.x', 'inv_dist1', 'tertiary_atom_2', 'molecule_atom_index_0_dist_max_div', 'tertiary_distance_2', 'tertiary_angle_0', 'dist_O_1_x', 'tertiary_distance_3', 'mean_molecule_atom_0_dist_xyz', 'inv_distPR', 'min_molecule_atom_0_dist_xyz', 'molecule_type_dist_max', 'dist_C_0_x', 'dist_H_1_x', 'adC1', 'dist_O_0_x', 'molecule_atom_index_0_dist_max_diff', 'adN1', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'inv_dist0', 'dist_C_3_x', 'atom_0_bond_lengths_max', 'atom_1_bond_lengths_min', 'bond_atom', 'inv_dist0R', 'dist_H_0_y', 'molecule_atom_index_1_dist_min_div', 'dist_C_2_x', 'cos_c0', 'dist_H_0_x', 'tertiary_distance_1', 'dist_H_2_y', 'tertiary_atom_3', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'vander_O.y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'yukawa_O.y', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_2', 'tertiary_atom_0', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'atom_1_bond_lengths_max', 'dist_C_1_y', 'distC0', 'adC2', 'tertiary_atom_4', 'cos_c0_c1', 'atom_index_1_n_cycle', 'dist_N_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_sd','oof_fc','oof_pso','tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_sd','oof_fc','oof_pso','tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_sd','oof_fc','oof_pso','tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_sd','oof_fc','oof_pso','tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_sd','oof_fc','oof_pso','max_molecule_atom_0_dist_xyz', 'tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_1_dist_min_diff', 'dist_C_4_x', 'adC3', 'distN0', 'atom_0_bond_lengths_max', 'dist_C_3_y', 'tertiary_angle_1', 'atom_1_bond_lengths_mean', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'molecule_atom_index_1_dist_max_div', 'molecule_atom_index_0_dist_min_div', 'dist_C_0_x', 'adC1', 'tertiary_distance_1', 'atom_1_bond_lengths_std', 'dist_H_0_y', 'dist_xyz', 'dist_O_0_x', 'inv_distPR', 'cos_c0_c1', 'cos_c0', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_C_3_x', 'atom_1_n_bonds', 'bond_atom', 'dist_C_2_x', 'dist_N_0_y', 'dist_N_0_x', 'atom_index_1_cycle_size_mean', 'min_molecule_atom_0_dist_xyz', 'tertiary_angle_3', 'tertiary_distance_3', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}]

In [11]:
df_train.type.unique().tolist()[-1:]

[7]

In [14]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist()[-1:]:
#     df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=False, trial_level=1)
    study = optuna.create_study()
    study.optimize(objective, n_trials=200)
    print(study.best_params)
#     df_his = pd.concat([df_his, df_his_i], axis=0)
#     df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
#     df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
#     df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

# df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
# df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

[I 2019-07-21 22:30:49,278] Finished trial#0 resulted in value: 0.15055486117172784. Current best value is 0.15055486117172784 with parameters: {'learning_rate': 0.19031456477956496, 'feature_fraction': 0.8042381951205915, 'bagging_fraction': 0.8479694074496851, 'min_data_in_leaf': 424, 'lambda_l1': 3.335186029357353e-05, 'lambda_l2': 2.466018563628591, 'max_bin': 42, 'num_leaves': 21}.
[I 2019-07-21 22:30:53,004] Finished trial#1 resulted in value: 0.159528719068809. Current best value is 0.15055486117172784 with parameters: {'learning_rate': 0.19031456477956496, 'feature_fraction': 0.8042381951205915, 'bagging_fraction': 0.8479694074496851, 'min_data_in_leaf': 424, 'lambda_l1': 3.335186029357353e-05, 'lambda_l2': 2.466018563628591, 'max_bin': 42, 'num_leaves': 21}.
[I 2019-07-21 22:30:57,865] Finished trial#2 resulted in value: 0.1490194058178609. Current best value is 0.1490194058178609 with parameters: {'learning_rate': 0.46039071153205535, 'feature_fraction': 0.8373891367634861, '

[I 2019-07-21 22:32:46,723] Finished trial#21 resulted in value: 0.1447677427287033. Current best value is 0.1225848411179872 with parameters: {'learning_rate': 0.2984920033778189, 'feature_fraction': 0.929823120508402, 'bagging_fraction': 0.6717057330274558, 'min_data_in_leaf': 334, 'lambda_l1': 1.3554915854587142, 'lambda_l2': 0.20412488872524406, 'max_bin': 94, 'num_leaves': 325}.
[I 2019-07-21 22:32:54,631] Finished trial#22 resulted in value: 0.13161140449071676. Current best value is 0.1225848411179872 with parameters: {'learning_rate': 0.2984920033778189, 'feature_fraction': 0.929823120508402, 'bagging_fraction': 0.6717057330274558, 'min_data_in_leaf': 334, 'lambda_l1': 1.3554915854587142, 'lambda_l2': 0.20412488872524406, 'max_bin': 94, 'num_leaves': 325}.
[I 2019-07-21 22:33:00,573] Finished trial#23 resulted in value: 0.14093242135873413. Current best value is 0.1225848411179872 with parameters: {'learning_rate': 0.2984920033778189, 'feature_fraction': 0.929823120508402, 'bag

[I 2019-07-21 22:34:58,149] Finished trial#42 resulted in value: 0.15062845109849704. Current best value is 0.11969792172206797 with parameters: {'learning_rate': 0.498696415132855, 'feature_fraction': 0.9812950236816153, 'bagging_fraction': 0.6272883527056892, 'min_data_in_leaf': 205, 'lambda_l1': 6.885405236711593e-06, 'lambda_l2': 14.791114557076948, 'max_bin': 39, 'num_leaves': 393}.
[I 2019-07-21 22:35:05,707] Finished trial#43 resulted in value: 0.13739296665295592. Current best value is 0.11969792172206797 with parameters: {'learning_rate': 0.498696415132855, 'feature_fraction': 0.9812950236816153, 'bagging_fraction': 0.6272883527056892, 'min_data_in_leaf': 205, 'lambda_l1': 6.885405236711593e-06, 'lambda_l2': 14.791114557076948, 'max_bin': 39, 'num_leaves': 393}.
[I 2019-07-21 22:35:14,433] Finished trial#44 resulted in value: 0.1333880837701979. Current best value is 0.11969792172206797 with parameters: {'learning_rate': 0.498696415132855, 'feature_fraction': 0.981295023681615

[I 2019-07-21 22:37:10,136] Finished trial#62 resulted in value: 0.12842115259191078. Current best value is 0.11969792172206797 with parameters: {'learning_rate': 0.498696415132855, 'feature_fraction': 0.9812950236816153, 'bagging_fraction': 0.6272883527056892, 'min_data_in_leaf': 205, 'lambda_l1': 6.885405236711593e-06, 'lambda_l2': 14.791114557076948, 'max_bin': 39, 'num_leaves': 393}.
[I 2019-07-21 22:37:18,427] Finished trial#63 resulted in value: 0.12031591051814343. Current best value is 0.11969792172206797 with parameters: {'learning_rate': 0.498696415132855, 'feature_fraction': 0.9812950236816153, 'bagging_fraction': 0.6272883527056892, 'min_data_in_leaf': 205, 'lambda_l1': 6.885405236711593e-06, 'lambda_l2': 14.791114557076948, 'max_bin': 39, 'num_leaves': 393}.
[I 2019-07-21 22:37:25,243] Finished trial#64 resulted in value: 0.12543649578654806. Current best value is 0.11969792172206797 with parameters: {'learning_rate': 0.498696415132855, 'feature_fraction': 0.98129502368161

[I 2019-07-21 22:39:48,950] Finished trial#82 resulted in value: 0.12870776880814558. Current best value is 0.11938686342170945 with parameters: {'learning_rate': 0.3641297054734397, 'feature_fraction': 0.9206735905800941, 'bagging_fraction': 0.6671746434338517, 'min_data_in_leaf': 234, 'lambda_l1': 8.321062060501192e-06, 'lambda_l2': 0.00040360396366852176, 'max_bin': 54, 'num_leaves': 475}.
[I 2019-07-21 22:39:56,654] Finished trial#83 resulted in value: 0.12932937769756664. Current best value is 0.11938686342170945 with parameters: {'learning_rate': 0.3641297054734397, 'feature_fraction': 0.9206735905800941, 'bagging_fraction': 0.6671746434338517, 'min_data_in_leaf': 234, 'lambda_l1': 8.321062060501192e-06, 'lambda_l2': 0.00040360396366852176, 'max_bin': 54, 'num_leaves': 475}.
[I 2019-07-21 22:40:04,998] Finished trial#84 resulted in value: 0.12277074553116205. Current best value is 0.11938686342170945 with parameters: {'learning_rate': 0.3641297054734397, 'feature_fraction': 0.920

...

[I 2019-07-21 22:44:33,672] Finished trial#122 resulted in value: 0.13196540506627147. Current best value is 0.11938686342170945 with parameters: {'learning_rate': 0.3641297054734397, 'feature_fraction': 0.9206735905800941, 'bagging_fraction': 0.6671746434338517, 'min_data_in_leaf': 234, 'lambda_l1': 8.321062060501192e-06, 'lambda_l2': 0.00040360396366852176, 'max_bin': 54, 'num_leaves': 475}.
[I 2019-07-21 22:44:41,214] Finished trial#123 resulted in value: 0.12545132999956388. Current best value is 0.11938686342170945 with parameters: {'learning_rate': 0.3641297054734397, 'feature_fraction': 0.9206735905800941, 'bagging_fraction': 0.6671746434338517, 'min_data_in_leaf': 234, 'lambda_l1': 8.321062060501192e-06, 'lambda_l2': 0.00040360396366852176, 'max_bin': 54, 'num_leaves': 475}.
[I 2019-07-21 22:44:50,329] Finished trial#124 resulted in value: 0.1263672694271183. Current best value is 0.11938686342170945 with parameters: {'learning_rate': 0.3641297054734397, 'feature_fraction': 0.9

[I 2019-07-21 22:46:59,392] Finished trial#142 resulted in value: 0.130008322665655. Current best value is 0.11938686342170945 with parameters: {'learning_rate': 0.3641297054734397, 'feature_fraction': 0.9206735905800941, 'bagging_fraction': 0.6671746434338517, 'min_data_in_leaf': 234, 'lambda_l1': 8.321062060501192e-06, 'lambda_l2': 0.00040360396366852176, 'max_bin': 54, 'num_leaves': 475}.
[I 2019-07-21 22:47:05,579] Finished trial#143 resulted in value: 0.1339453909606005. Current best value is 0.11938686342170945 with parameters: {'learning_rate': 0.3641297054734397, 'feature_fraction': 0.9206735905800941, 'bagging_fraction': 0.6671746434338517, 'min_data_in_leaf': 234, 'lambda_l1': 8.321062060501192e-06, 'lambda_l2': 0.00040360396366852176, 'max_bin': 54, 'num_leaves': 475}.
[I 2019-07-21 22:47:14,708] Finished trial#144 resulted in value: 0.13128066040313174. Current best value is 0.11938686342170945 with parameters: {'learning_rate': 0.3641297054734397, 'feature_fraction': 0.920

[I 2019-07-21 22:49:51,112] Finished trial#162 resulted in value: 0.12456255576915279. Current best value is 0.11938686342170945 with parameters: {'learning_rate': 0.3641297054734397, 'feature_fraction': 0.9206735905800941, 'bagging_fraction': 0.6671746434338517, 'min_data_in_leaf': 234, 'lambda_l1': 8.321062060501192e-06, 'lambda_l2': 0.00040360396366852176, 'max_bin': 54, 'num_leaves': 475}.
[I 2019-07-21 22:50:00,214] Finished trial#163 resulted in value: 0.11951223183039042. Current best value is 0.11938686342170945 with parameters: {'learning_rate': 0.3641297054734397, 'feature_fraction': 0.9206735905800941, 'bagging_fraction': 0.6671746434338517, 'min_data_in_leaf': 234, 'lambda_l1': 8.321062060501192e-06, 'lambda_l2': 0.00040360396366852176, 'max_bin': 54, 'num_leaves': 475}.
[I 2019-07-21 22:50:07,257] Finished trial#164 resulted in value: 0.12442250305114902. Current best value is 0.11938686342170945 with parameters: {'learning_rate': 0.3641297054734397, 'feature_fraction': 0.

[I 2019-07-21 22:52:36,792] Finished trial#182 resulted in value: 0.11963388530555559. Current best value is 0.11874693685908375 with parameters: {'learning_rate': 0.3901303601603288, 'feature_fraction': 0.9749846228880419, 'bagging_fraction': 0.6926257454153049, 'min_data_in_leaf': 201, 'lambda_l1': 4.678724334685024e-06, 'lambda_l2': 6.180574870766062e-06, 'max_bin': 46, 'num_leaves': 398}.
[I 2019-07-21 22:52:44,055] Finished trial#183 resulted in value: 0.1265013869675155. Current best value is 0.11874693685908375 with parameters: {'learning_rate': 0.3901303601603288, 'feature_fraction': 0.9749846228880419, 'bagging_fraction': 0.6926257454153049, 'min_data_in_leaf': 201, 'lambda_l1': 4.678724334685024e-06, 'lambda_l2': 6.180574870766062e-06, 'max_bin': 46, 'num_leaves': 398}.
[I 2019-07-21 22:52:52,024] Finished trial#184 resulted in value: 0.12412058920819244. Current best value is 0.11874693685908375 with parameters: {'learning_rate': 0.3901303601603288, 'feature_fraction': 0.974

{'learning_rate': 0.3901303601603288, 'feature_fraction': 0.9749846228880419, 'bagging_fraction': 0.6926257454153049, 'min_data_in_leaf': 201, 'lambda_l1': 4.678724334685024e-06, 'lambda_l2': 6.180574870766062e-06, 'max_bin': 46, 'num_leaves': 398}


In [ ]:
"""
{'learning_rate': 0.4840263887834881, 'feature_fraction': 0.9264366442345999, 'bagging_fraction': 0.7870522973541787, 'min_data_in_leaf': 638, 'lambda_l1': 5.399937304152897, 'lambda_l2': 33.21329780071299, 'max_bin': 66, 'num_leaves': 511}
{'learning_rate': 0.47664150293254715, 'feature_fraction': 0.9345515622882533, 'bagging_fraction': 0.9471105845283242, 'min_data_in_leaf': 325, 'lambda_l1': 0.027167889983129943, 'lambda_l2': 65.04871529054178, 'max_bin': 90, 'num_leaves': 492}
{'learning_rate': 0.4788084435748641, 'feature_fraction': 0.9361895362210686, 'bagging_fraction': 0.7545999456089862, 'min_data_in_leaf': 200, 'lambda_l1': 8.081922325265403e-06, 'lambda_l2': 0.96398554317009, 'max_bin': 73, 'num_leaves': 495}
{'learning_rate': 0.34872110827299774, 'feature_fraction': 0.9610608076419638, 'bagging_fraction': 0.6262039284042835, 'min_data_in_leaf': 200, 'lambda_l1': 1.0238214451995545e-05, 'lambda_l2': 6.373180568952514, 'max_bin': 82, 'num_leaves': 427}
{'learning_rate': 0.45605588487213117, 'feature_fraction': 0.9841034261836108, 'bagging_fraction': 0.7063253284308669, 'min_data_in_leaf': 394, 'lambda_l1': 4.898389805875972, 'lambda_l2': 79.39455452813532, 'max_bin': 82, 'num_leaves': 502}
{'learning_rate': 0.437459170449037, 'feature_fraction': 0.9829680390216893, 'bagging_fraction': 0.8960507398136508, 'min_data_in_leaf': 310, 'lambda_l1': 9.880462565093161e-06, 'lambda_l2': 5.3187569224419286e-05, 'max_bin': 95, 'num_leaves': 503}
{'learning_rate': 0.3901303601603288, 'feature_fraction': 0.9749846228880419, 'bagging_fraction': 0.6926257454153049, 'min_data_in_leaf': 201, 'lambda_l1': 4.678724334685024e-06, 'lambda_l2': 6.180574870766062e-06, 'max_bin': 46, 'num_leaves': 398}
"""

In [15]:
param_model_init_list = [
    {'learning_rate': 0.4840263887834881, 'feature_fraction': 0.9264366442345999, 'bagging_fraction': 0.7870522973541787, 'min_data_in_leaf': 638, 'lambda_l1': 5.399937304152897, 'lambda_l2': 33.21329780071299, 'max_bin': 66, 'num_leaves': 511},
{'learning_rate': 0.47664150293254715, 'feature_fraction': 0.9345515622882533, 'bagging_fraction': 0.9471105845283242, 'min_data_in_leaf': 325, 'lambda_l1': 0.027167889983129943, 'lambda_l2': 65.04871529054178, 'max_bin': 90, 'num_leaves': 492},
{'learning_rate': 0.4788084435748641, 'feature_fraction': 0.9361895362210686, 'bagging_fraction': 0.7545999456089862, 'min_data_in_leaf': 200, 'lambda_l1': 8.081922325265403e-06, 'lambda_l2': 0.96398554317009, 'max_bin': 73, 'num_leaves': 495},
{'learning_rate': 0.34872110827299774, 'feature_fraction': 0.9610608076419638, 'bagging_fraction': 0.6262039284042835, 'min_data_in_leaf': 200, 'lambda_l1': 1.0238214451995545e-05, 'lambda_l2': 6.373180568952514, 'max_bin': 82, 'num_leaves': 427},
{'learning_rate': 0.45605588487213117, 'feature_fraction': 0.9841034261836108, 'bagging_fraction': 0.7063253284308669, 'min_data_in_leaf': 394, 'lambda_l1': 4.898389805875972, 'lambda_l2': 79.39455452813532, 'max_bin': 82, 'num_leaves': 502},
{'learning_rate': 0.437459170449037, 'feature_fraction': 0.9829680390216893, 'bagging_fraction': 0.8960507398136508, 'min_data_in_leaf': 310, 'lambda_l1': 9.880462565093161e-06, 'lambda_l2': 5.3187569224419286e-05, 'max_bin': 95, 'num_leaves': 503},
{'learning_rate': 0.3901303601603288, 'feature_fraction': 0.9749846228880419, 'bagging_fraction': 0.6926257454153049, 'min_data_in_leaf': 201, 'lambda_l1': 4.678724334685024e-06, 'lambda_l2': 6.180574870766062e-06, 'max_bin': 46, 'num_leaves': 398}
]

In [16]:
params_list=[]
for param, model_init in zip(columns_list, param_model_init_list):
    param['model']['init'] = model_init
    params_list.append(param)

In [ ]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), params_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=False, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [ ]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'message']]

In [ ]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

In [37]:
df_test_pred.tail()

,index,0,1,2,3,4
2505537,7163684,0.325547,0.396931,0.603638,0.624082,0.754078
2505538,7163685,3.296284,3.946417,3.698317,2.891865,3.334881
2505539,7163686,2.460081,2.764959,2.995366,1.896396,2.637677
2505540,7163687,4.851247,4.834557,4.938143,4.788908,4.589528
2505541,7163688,116.324707,115.820266,117.051927,116.287453,116.206033


In [38]:
idx=0.38
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['scalar_coupling_constant'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_csv('../../data/submission/submission_lgbm_{}.csv'.format(idx), index=False)